In [3]:
# importing

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

In [30]:
from IPython.display import HTML
HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

# Interactive Dashboard: Covid-19 

In [22]:
# load data
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

## Confirmed Cases

[full confirmed list](confirmed_cases.ipynb)

In [23]:
confirmed_df.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37162,37269,37345,37424,37431,37551,37596,37599,37599,37599
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6536,6676,6817,6971,7117,7260,7380,7499,7654,7812
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,35712,36204,36699,37187,37664,38133,38583,39025,39444,39847
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,963,963,977,981,989,989,989,1005,1005,1024
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1679,1735,1762,1815,1852,1879,1906,1935,1966,2015


## Death Cases

[full death list](death_cases.ipynb)

In [24]:
death_df.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1328,1344,1354,1363,1363,1370,1375,1375,1375,1375
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,200,205,208,213,219,225,228,230,232,234
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1312,1322,1333,1341,1351,1360,1370,1379,1391,1402
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,52,52,53,53,53,53,53,53,53,53
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,78,80,80,80,86,86,88,88,90,92


## Recovered Cases

[full recovered list](recovered_cases.ipynb)

In [26]:
recovered_df.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,26228,26415,26694,26714,26714,27166,27166,27166,27166,27166
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3379,3480,3552,3616,3695,3746,3794,3816,3871,3928
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,24920,25263,25627,26004,26308,26644,27017,27347,27653,27971
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,839,839,855,858,863,863,863,869,869,875
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,569,575,577,577,584,628,628,632,667,698


## Cases of Each Country

[full country list](country_cases.ipynb)

In [27]:
country_df.head(5)

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
0,Afghanistan,2020-08-21 02:27:40,33.93911,67.709953,37856.0,1385.0,27681.0,8790.0,97.245346,NaN,NaN,3.658601,4,AFG
1,Albania,2020-08-21 02:27:40,41.15330,20.168300,7967.0,238.0,3986.0,3743.0,276.843422,NaN,NaN,2.987323,8,ALB
2,Algeria,2020-08-21 02:27:40,28.03390,1.659600,40258.0,1411.0,28281.0,10566.0,91.806254,NaN,NaN,3.504893,12,DZA
3,Andorra,2020-08-21 02:27:40,42.50630,1.521800,1024.0,53.0,875.0,96.0,1325.309001,NaN,NaN,5.175781,20,AND
4,Angola,2020-08-21 02:27:40,-11.20270,17.873900,2044.0,93.0,742.0,1209.0,6.219142,NaN,NaN,4.549902,24,AGO


In [28]:
# data cleaning - renaming
country_df.columns = map(str.lower, country_df.columns)
recovered_df.columns = map(str.lower,recovered_df.columns)
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower,confirmed_df.columns)

In [29]:
confirmed_df = confirmed_df.rename(columns = {'province/state':'state','country/region':'country'})
recovered_df = recovered_df.rename(columns = {'province/state':'state','country/region':'country'})
death_df = death_df.rename(columns = {'province/state':'state','country/region':'country'})
country_df = country_df.rename(columns = {'country_region':'country'})
                                            
                                            

In [11]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [12]:
# displaying the total stats

display(HTML("<div style = 'background-color: #5499C7; padding: 55px; border-style = 'solid' '>" +
             "<span style='color: #fff; font-size:35px;font-family:serif'> Confirmed: "  + str(confirmed_total) +"</span>" +
             "<span style='color: #E74C3C; font-size:35px;margin-left:20px;font-family:serif'> Deaths: " + str(deaths_total) + "</span>"+
             "<span style='color: #2ECC71; font-size:35px; margin-left:20px;font-family:serif'> Recovered: " + str(recovered_total) + "</span>"+
             "</div>")
       )


## Global Map: The Spread of Covid-19
#### Hover your mouse on a circle to that country's case amounts

In [13]:
world_map = folium.Map(location = [11,0], tiles = 'cartodbpositron', zoom_start = 2,
                     max_zoom = 6, min_zoom = 2)

for i in range(len(confirmed_df)):
    folium.Circle(
    location = [confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
    fill = True,
    radius = (int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2) * 50000,
    fill_color = '#3498DB',
    color = "red",
    tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
    ).add_to(world_map)

world_map


<img src="Covid_19Fremont.gif" width="850" align="center">

In [14]:
sorted_country_df = country_df.sort_values('confirmed', ascending = False).head(10)

## View the details of the worst affected countries
#### Enter the number of countries that you want data for

In [15]:
# sorting the values by confirmed descednding order
# country_df.sort_values('confirmed', ascending= False).head(10).style.background_gradient(cmap='copper')
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    g = 'background-color: grey'
    gr = 'background-color: green'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = g
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = gr
    
    return df1

def show_latest_cases(number):
    number = int(number)
    return country_df.sort_values('confirmed', ascending= False).head(number).style.apply(highlight_col, axis=None)

interact(show_latest_cases, number='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='number'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

#### Adjust the slider to the number of countries you want data for

In [16]:
## plotting the worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 1100
    )
    fig.show();

interact(bubble_chart, n=10);

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

## View the details of a specific country or the world
#### Enter the country name and capitalize the first letter

In [17]:
import plotly.graph_objects as go

def plot_cases_for_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue','red']
    mode_size = [6,8]
    line_size = [4,5]

    df_list = [confirmed_df,death_df]

    fig = go.Figure()

    for i, df in enumerate(df_list):
        if country == "World" or country == 'world':
            x_data = np.array(list(df.iloc[:,5:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,5:]), axis = 0)
            
        else:
            x_data = np.array(list(df.iloc[:,5:].columns))
            y_data = np.sum(np.asarray(df[df['country']==country].iloc[:,5:]),axis = 0)
        
        fig.add_trace(go.Scatter(x = x_data, y = y_data, mode = 'lines+markers',
                                name = labels[i],
                                line = dict(color = colors[i], width = line_size[i]),
                                connectgaps = True,
                                text = "Total " + str(labels[i]) + ': ' + str(y_data[-1])
                                ))
    fig.show()
        
#plot_cases_for_country('Brazil')

interact(plot_cases_for_country, country='World');
        

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

## Bar Charts: Top Ten Worst Affected Countries

In [18]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Confirmed Cases", # the axis names
    color_discrete_sequence=["lightgrey"], 
    height=500,
    width=800
)

In [19]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Death Cases", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [20]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "recovered",
    title= "Recovered Cases", # the axis names
    color_discrete_sequence=["lightgreen"], 
    height=500,
    width=800
)